# Single Neuron Biophysics notebook

*David Sterratt, October 2025*

This notebook accompanies the MSc Neuroscience lecture on Single Neuron Biophysics

## Part 1: Simulating how voltage changes over time

### 1.A The first simulation

We are going to write code to simulate how the voltage changes across the membrane of a cell in which there are almost equal concentrations of positive and negative ions on either side, but with a slight excess of positive charges on the inside, which means that the membrane potential $V$ is positive. The cell membrane has a capacitance of 1 picofarad, i.e. $C=1\times 10^{-12}$F and a resistance of 10 gigaohms, i.e. $R=10\times10^9\Omega$.

We will first set the parameters. Put your cursor in this cell and press "Shift" and "Return" together.

In [1]:
# Anything after a hash sign (#) is a comment - it will not do 

C = 1E-12 # farads
R = 10E9  # ohms

You can now check the value of C and R using the print command.

Run the cell below with Shift+Return.

In [2]:
print(C)
print(R)

1e-12
10000000000.0


We decide that we'll be advancing time in 1 millisecond at a time, so $\Delta t=1$ millisecond   $= 1\times 10^{-3}$ seconds. By convention, we'll refer to $\Delta t=1$ as `dt`.

Run the cell below with Shift+Return (and do this from now on, unless we ask you to do something else).

In [3]:
dt = 1E-3 # seconds

Then we will initialise the time $t$ to 0 and the voltage $V$ to 5mV, and print them out in a prettier format than above.

In [4]:
t = 0    # seconds
V = 5E-3 # volts
print('t = %0.3fs ; V = %2.5fV'%(t, V)) 

t = 0.000s ; V = 0.00500V


We'll now advance time forward by one step using the formula 
$$V(t+\Delta t) = V(t) - \frac{V(t)}{RC}\Delta t$$
and print out the new voltage.

However, we want you to
1. fill in the step update step and
2. run this cell with Ctrl+Return, which will mean that the cursor stays in the cell.
3. Keep pressing Ctrl+Return, and see that happens to the voltage  

In [5]:
t = t + dt
V = V - V/(R*C)*dt # Delete
print('t = %0.3fs ; V = %2.5fV'%(t, V)) 

t = 0.001s ; V = 0.00450V


Well done! You've run a simulation. Technically speaking you've solved the simulation using Euler integration.

## 1.B Saving and visualising the output

We can see that the membrane potential decays in the simulation above, but it would be good (a) to visualise it and (b) instead of clicking, automate stepping through time.

We're going to
1. Create some __arrays__ to store the values of $V$ and $t$.
2. Iterate the update procedure

In [48]:
import numpy as np                   # A module that we need to create arrays

# Parameters
C = 1E-12 # farads
R = 10E9  # ohms

# Simulation control
N = 500          # Number of steps we want to run for
dt = 1E-3         # Time step

# Space to store output
t = np.zeros(N+1) # Array with N+1 elements where we will save time
V = np.zeros(N+1) # Array with N+1 elements where we will save membrane potential

# Initialisation
t[0] = 0
V[0] = 5E-3

# Repeat the update step N times
for i in range(N):
    t[i+1] = t[i] + dt
    V[i+1] = V[i] - V[i]/(R*C)*dt

Now we'll plot - don't worry about the code below; just press Shift+Enter.

In [49]:
from bokeh.io import output_notebook # A module that we need to plot 
import bokeh.plotting as plt         # A module that we need to plot 
output_notebook()

f = plt.figure(x_axis_label="t (s)", y_axis_label="v (V)", aspect_ratio=2)
f.line(t, V, line_width=2)
plt.show(f)

Loading BokehJS ...

Questions:

1. How long does it take for the voltage to reduce to about 1/3 of the way to 0? (You can zoom in)
2. Try increasing R by a factor of 5. How slow is the decrease now?
3. Try increasing C by a factor of 2. How slow is the decrease now?
4. What is the product of $R$ and $C$? Does it relate to the time of the decrease?
5. Try increasing the timestep by a factor of 10 and decreasing N by a factor of 10
6. Try the same thing again!


## Part 2: Using the NEURON simulation library

The code above works for the very simple example, and could be extended further. However it is also not very efficient, and specifying more complex models would require a lot of code. Modelling environments like the NEURON simulation library are designed to be very efficient, and to make model specification (fairly) intuitive.

We'll start by installing the package. Press Shift+Return in the cell below, and wait for it finish, when it will say "Successfully installed...".

In [8]:
!pip install neuron

  Using cached neuron-9.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.5 kB)
  Using cached find_libpython-0.4.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached setuptools-80.8.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
Using cached neuron-9.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (18.1 MB)
Using cached find_libpython-0.4.0-py3-none-any.whl (8.7 kB)
Using cached setuptools-80.8.0-py3-none-any.whl (1.2 MB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [sympy]
      Successfully uninstalled sympy-1.14.0━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [sympy]
  Attempting uninstall: setuptools90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [sympy]
    Found existing installation: setuptools 80.9.0━━━━━━━━━━━━ 1/4 [sympy]
    Uninstalling setuptools-80.9.0:━━━━━━━━━━

Now import the relevant parts of NEURON. Don't worry about the warning about the DISPLAY variable.

In [9]:
from neuron import n
from neuron.units import ms, mV
n.load_file("stdrun.hoc")

--No graphics will be displayed.


1.0

### 2.A Create and run a cell model

We will first create the simplest possible model of a patch of
membrane: a single passive compartment.

![img](schematic-cir-RC.png)

The equivalent electrical
circuit comprises a membrane capacitance $C_\text{m}$, a membrane
resistance $R_\text{m}$, a battery whose electromotive force is the same as
the resting membrane potential $V_\text{m}$ and a current source $I_\text{e}$.  We
will imagine our circuit represents a soma, though in fact it misses
some crucial features of a real soma, namely active properties.

First we'll create a soma and print its default priorities:

In [10]:
# Create the soma
soma = n.Section("soma")
soma.psection()

{'point_processes': {},
 'density_mechs': {},
 'ions': {},
 'morphology': {'L': 100.0,
  'diam': [500.0],
  'pts3d': [],
  'parent': None,
  'trueparent': None},
 'nseg': 1,
 'Ra': 35.4,
 'cm': [1.0],
 'regions': set(),
 'species': set(),
 'name': 'soma',
 'hoc_internal_name': '__nrnsec_0x5568ff75e2c0',
 'cell': None}

We don't need to know what all of these are, but some are:
- `cm` - specific capacitance in µF/cm<sup>-2</sup>
- `L` - length of the "section" in µm, which is a cylinder
- `diam` - diameter of the "section" in µm

Note there is no membrane resistance mentioned; we will introduce this by inserting a "passive" mechanism.  NEURON has a special channel type called `pas` to
simulate the membrane resistance and reversal potential. To insert it into the membrane, type:

In [11]:
# Insert passive channels
soma.insert(n.pas)
soma.psection()

{'point_processes': {},
 'density_mechs': {'pas': {'g': [0.001], 'e': [-70.0], 'i': [0.0]}},
 'ions': {},
 'morphology': {'L': 100.0,
  'diam': [500.0],
  'pts3d': [],
  'parent': None,
  'trueparent': None},
 'nseg': 1,
 'Ra': 35.4,
 'cm': [1.0],
 'regions': set(),
 'species': set(),
 'name': 'soma',
 'hoc_internal_name': '__nrnsec_0x5568ff75e2c0',
 'cell': None}

Now we can see a `pas` in `density_mechs`. It has:
- `g` specific membrane conductance $g_\text{m}$ in S/cm<sup>2</sup>. As conductance is the inverse of resistance, the value 0.001S/cm<sup>2</sup> is equivalent to a specific membrane resistance of 1000Ωcm<sup>2</sup>
- `e` the passive associated reversal potential $E_\text{m}$ in mV
- `i` the current presently flowing though the passive mechanism in mA/cm<sup>2</sup>

We can now run the simulation with the code below. What would you expect to see?

In [60]:
# Set up recordings of the membrane potential and time
v = n.Vector().record(soma(0.5)._ref_v)      # Membrane potential vector
i_pas = n.Vector().record(soma(0.5)._ref_i_pas)  # Current through passive mechanism
i_cap = n.Vector().record(soma(0.5)._ref_i_cap)  # Current through passive mechanism
t = n.Vector().record(n._ref_t)              # Time stamp vector

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(40 * ms)

0.0

In [61]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)


We should see the same sort of behaviour as in the simulation we wrote earlier.

A more typical value for the specific membrane resistance is $R_\text{m} = 10$Ωcm<sup>2</sup>. This means that the specific
membrane conductance should be 0.0001 S/cm<sup>2</sup>. Run the cell below to change the value of the membrane conductance and re-run the simulation. 

In [62]:
# Set the membrane conductance to 0.0001 S/cm2
soma(0.5).pas.g = 0.0001

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(40 * ms)

# Plot
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)

### Injecting current into a single compartment

We have now built a single compartment with passive channels. To test its properties, we will inject some current. To do this, run this cell:

In [63]:
# Insert a current clamp electrode in the centre of the soma
stim = n.IClamp(soma(0.5))
stim.amp = 1
stim.delay = 100
stim.dur = 100

This will inject a pulse of current with an amplitude of 1 nA (`stim.amp`) starting 100 ms (`stim.delay`) after the simulation has
started and lasting for 100 ms (`stim.dur`).  We will want to run the simulation for longer, to see the full effect.

In [64]:
# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 200 milliseconds
n.continuerun(250 * ms)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)


We can also examine the current flowing through the membrane (blue) and the current flowing onto the membrane (red). Remember that positive current is outwards. 

In [66]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="i (mA)", aspect_ratio=2)
f.line(t, i_pas, line_width=2)
f.line(t, i_cap, line_width=2, color='red')

plt.show(f)

Try to explain what you're seeing to yourself. Then look at the explanation below.

### Explanation

The voltage first settles to −70 mV (the resting membrane potential) from its starting value of −65 mV. Then, at 100 ms (the time of the current
injection onset), it starts to rise, though not instantaneously. This is because when current is injected it will first mainly charge the membrane capacitance, but as this charges up, more of it will flow through
the membrane resistance. It levels out at about −63.65 mV. In this steady state, all the current flows
through the membrane resistance rather than onto the membrane capacitance.
At 200 ms (the time of the current offset) the membrane potential starts to decay back to the resting
potential. This is due to the capacitance discharging slowly through the resistance. This is a bit like a
battery going flat, though on a much faster timescale!
We can check that the simulation is doing what it should be by making measurements of the simulated
results and then comparing them to the theoretical values of the same measurements.


#### The input resistance
The input resistance $R_\text{in}$ is a measure of how much current needs to be injected to increase the membrane
potential by a certain amount, and is often measured experimentally. The input resistance is defined

$$R_{\text{in}} = \frac{\Delta V}{\Delta i_\text{e}}$$

where $\Delta V$ is a small change in membrane potential due to a small change in injection current $\Delta i_{\text{e}}$. In
the case of voltage-dependent conductances, the input resistance may vary depending on the holding
membrane potential it is measured at. In our case, the conductances are fixed, so the input resistance
will not depend on the holding membrane potential and we do not need to make our change in current
and membrane potential small.

# Part 3: 